In [ ]:
# 1. Importar librerías
import ipaddress
import re
import urllib.request
from bs4 import BeautifulSoup
import socket
import requests
import google
import whois
from datetime import date, datetime
import time
from dateutil.parser import parse as date_parse
from urllib.parse import urlparse

In [ ]:
# 1.  Definir la clase
class ExtraccionUrl:
    features = []
    def __init__(self,url):
        self.features = []
        self.url = url
        self.domain = ""
        self.whois_response = ""
        self.urlparse = ""
        self.response = ""
        self.soup = ""
        
        try:
            self.response = requests.get(url)
            self.soup = BeautifulSoup(response.text, 'html.parser')
        except:
            pass
        
        try:
            self.urlparse = urlparse(url)
            self.domain = self.urlparse.netloc
        except:
            pass
        
        try:
            self.whois_response = whois.whois(self.domain)
        except:
            pass
        
        self.features.append(self.UsoIP())
        self.features.append(self.UrlLargo())
        self.features.append(self.URLCorto())
        self.features.append(self.SimboloArroba())
        self.features.append(self.Redireccion())
        self.features.append(self.PrefijoSufijo())
        self.features.append(self.SubDominio())
        self.features.append(self.Https())
        self.features.append(self.ExpiracionCreacionDominio())
        self.features.append(self.Icono())
        
        self.features.append(self.NoPuertoStd())
        self.features.append(self.HTTPSURLDominio())
        self.features.append(self.URLRespuesta())
        self.features.append(self.URLAnclaje())
        self.features.append(self.ScriptEnlaceEnEtiquetas())
        self.features.append(self.ControladorFormulariosServidor())
        self.features.append(self.InformacionCorreo())
        self.features.append(self.URLAnormal())
        self.features.append(self.ReenvioSitiosWeb())
        self.features.append(self.BarraEstadoPersonalizado())

        self.features.append(self.DeshabilitacionClicDerecho())
        self.features.append(self.UsoVentanaEmergente())
        self.features.append(self.RedireccionMarcoFlotante())
        self.features.append(self.EdadDeDominio())
        self.features.append(self.GrabacionDNS())
        self.features.append(self.TraficoSitioWeb())
        self.features.append(self.RangoPagina())
        self.features.append(self.IndiceGoogle())
        self.features.append(self.EnlacesApuntanPagina())
        self.features.append(self.ReporteEstadisticas())
        
        # 1.UsoIP
    def UsoIP(self):
        try:
            ipaddress.ip_address(self.url)
            return -1
        except:
            return 1

    # 2.UrlLargo
    def UrlLargo(self):
        if len(self.url) < 54:
            return 1
        elif len(self.url) in range(54, 76):
            return 0
        else:
            return -1

    # 3.URLCorto
    def URLCorto(self):
        match = re.search('bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|'
                    'yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|'
                    'short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|'
                    'doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|'
                    'db\.tt|qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|'
                    'q\.gs|is\.gd|po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|'
                    'x\.co|prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|v\.gd|tr\.im|link\.zip\.net', self.url)
        if match:
            return -1
        return 1

    # 4.SimboloArroba
    def SimboloArroba(self):
        if re.findall("@",self.url):
            return -1
        return 1
    
    # 5.Redireccion//
    def Redireccion(self):
        if self.url.rfind('//')>6:
            return -1
        return 1
    
    # 6.PrefijoSufijo
    def PrefijoSufijo(self):
        try:
            match = re.findall('\-', self.domain)
            if match:
                return -1
            return 1
        except:
            return -1
    
    # 7.SubDominio
    def SubDominio(self):
        dot_count = len(re.findall("\.", self.url))
        if dot_count == 1:
            return 1
        elif dot_count == 2:
            return 0
        return -1

    # 8.Https
    def Https(self):
        try:
            https = self.urlparse.scheme
            if 'https' in https:
                return 1
            return -1
        except:
            return 1

    # 9.ExpiracionCreacionDominio
    def ExpiracionCreacionDominio(self):
        try:
            expiration_date = self.whois_response.expiration_date
            creation_date = self.whois_response.creation_date
            try:
                if(len(expiration_date)):
                    expiration_date = expiration_date[0]
            except:
                pass
            try:
                if(len(creation_date)):
                    creation_date = creation_date[0]
            except:
                pass

            age = (expiration_date.year-creation_date.year)*12+ (expiration_date.month-creation_date.month)
            if age >=12:
                return 1
            return -1
        except:
            return -1

    # 10. Icono
    def Icono(self):
        try:
            for head in self.soup.find_all('head'):
                for head.link in self.soup.find_all('link', href=True):
                    if head.link['href'].endswith(('.png', '.ico')):
                        if self.url in head.link['href'] or domain in head.link['href']:
                            return 1
                        break
        except:
            pass
        finally:
            return -1

    # 11. NoPuertoStd
    def NoPuertoStd(self):
        try:
            port = self.domain.split(":")
            if len(port)>1:
                return -1
            return 1
        except:
            return -1

    # 12. HTTPSURLDominio
    def HTTPSURLDominio(self):
        try:
            if 'https' in self.domain:
                return -1
            return 1
        except:
            return -1
    
    # 13. URLRespuesta
    def URLRespuesta(self):
        multimedia_tags = ['img', 'audio', 'video', 'embed', 'iframe']
        success = 0
        total = 0
        try:
            for tag in multimedia_tags:
                elements = self.soup.find_all(tag, src=True)
                total += len(elements)
                for element in elements:
                    src = element['src']
                    if self.url in src or self.domain in src:
                        success += 1
                    else:
                        try:
                            parsed = urllib.parse.urlparse(src)
                            netloc = parsed.netloc
                            if self.domain in netloc:
                                success += 1
                        except:
                            pass

            if total > 0:
                percentage = success / float(total) * 100
                if percentage < 22.0:
                    return 1
                elif percentage < 61.0:
                    return 0
                else:
                    return -1
            else:
                return -1
        except:
            return -1
    
    # 14. URLAnclaje
    def URLAnclaje(self):
        try:
            unsafe = sum(a['href'].startswith(("#", "javascript", "mailto")) or not (url in a['href'] or self.domain in a['href']) for a in self.soup.find_all('a', href=True))
            i = len(self.soup.find_all('a', href=True))
            percentage = unsafe / i * 100.0
        except:
            return -1
        else:
            return 1 if percentage < 31.0 else 0 if percentage < 67.0 else -1

    # 15. ScriptEnlaceEnEtiquetas
    def ScriptEnlaceEnEtiquetas(self):
        MIN_PERCENTAGE = 17.0
        MAX_PERCENTAGE = 81.0
        try:
            total_links = 0
            valid_links = 0
            for link in self.soup.find_all('link', href=True):
                total_links = total_links + 1
                if es_enlace_valido(link['href']):
                    valid_links = valid_links + 1
            for script in self.soup.find_all('script', src=True):
                total_links = total_links + 1
                if es_enlace_valido(script['src']):
                    valid_links = valid_links + 1
            percentage = valid_links / float(total_links) * 100
            if percentage < MIN_PERCENTAGE:
                return 1
            elif percentage >= MIN_PERCENTAGE and percentage < MAX_PERCENTAGE:
                return 0
            else:
                return -1
        except:
            return -1

    # 16. ControladorFormulariosServidor
    def ControladorFormulariosServidor(self):
        try:
            if len(self.soup.find_all('form', action=True))==0:
                return 1
            else :
                for form in self.soup.find_all('form', action=True):
                    if form['action'] == "" or form['action'] == "about:blank":
                        return -1
                    elif self.url not in form['action'] and self.domain not in form['action']:
                        return 0
                    else:
                        return 1
        except:
            return -1

    # 17. InformacionCorreo
    def InformacionCorreo(self):
        try:
            if re.findall(r"[mail\(\)|mailto:?]", self.soap):
                return -1
            else:
                return 1
        except:
            return -1

    # 18. URLAnormal
    def URLAnormal(self):
        try:
            if self.response.text == self.whois_response:
                return 1
            else:
                return -1
        except:
            return -1

    # 19. ReenvioSitiosWeb
    def ReenvioSitiosWeb(self):
        try:
            if len(self.response.history) <= 1:
                return 1
            elif len(self.response.history) <= 4:
                return 0
            else:
                return -1
        except:
             return -1

    # 20. BarraEstadoPersonalizado
    def BarraEstadoPersonalizado(self):
        try:
            if re.findall("<script>.+onmouseover.+</script>", self.response.text):
                return 1
            else:
                return -1
        except:
             return -1

    # 21. DeshabilitacionClicDerecho
    def DeshabilitacionClicDerecho(self):
        try:
            if re.findall(r"event.button ?== ?2", self.response.text):
                return 1
            else:
                return -1
        except:
             return -1

    # 22. UsoVentanaEmergente
    def UsoVentanaEmergente(self):
        try:
            if re.findall(r"alert\(", self.response.text):
                return 1
            else:
                return -1
        except:
             return -1

    # 23. RedireccionMarcoFlotante
    def RedireccionMarcoFlotante(self):
        try:
            if re.findall(r"[<iframe>|<frameBorder>]", self.response.text):
                return 1
            else:
                return -1
        except:
             return -1

    # 24. EdadDeDominio
    def EdadDeDominio(self):
        try:
            creation_date = self.whois_response.creation_date
            try:
                if(len(creation_date)):
                    creation_date = creation_date[0]
            except:
                pass

            today  = date.today()
            age = (today.year-creation_date.year)*12+(today.month-creation_date.month)
            if age >=6:
                return 1
            return -1
        except:
            return -1

    # 25. GrabacionDNS    
    def GrabacionDNS(self):
        try:
            creation_date = self.whois_response.creation_date
            try:
                if(len(creation_date)):
                    creation_date = creation_date[0]
            except:
                pass

            today  = date.today()
            age = (today.year-creation_date.year)*12+(today.month-creation_date.month)
            if age >=6:
                return 1
            return -1
        except:
            return -1

    # 26. TraficoSitioWeb   
    def TraficoSitioWeb(self):
        try:
            rank = BeautifulSoup(urllib.request.urlopen("http://data.alexa.com/data?cli=10&dat=s&url=" + url).read(), "xml").find("REACH")['RANK']
            if (int(rank) < 100000):
                return 1
            return 0
        except :
            return -1

    # 27. RangoPagina
    def RangoPagina(self):
        try:
            prank_checker_response = requests.post("https://www.checkpagerank.net/index.php", {"name": self.domain})

            global_rank = int(re.findall(r"Global Rank: ([0-9]+)", rank_checker_response.text)[0])
            if global_rank > 0 and global_rank < 100000:
                return 1
            return -1
        except:
            return -1
            

    # 28. IndiceGoogle
    def IndiceGoogle(self):
        try:
            site = search(self.url, 5)
            if site:
                return 1
            else:
                return -1
        except:
            return 1

    # 29. EnlacesApuntanPagina
    def EnlacesApuntanPagina(self):
        try:
            number_of_links = len(re.findall(r"<a href=", self.response.text))
            if number_of_links == 0:
                return 1
            elif number_of_links <= 2:
                return 0
            else:
                return -1
        except:
            return -1

    # 30. ReporteEstadisticas
    def ReporteEstadisticas(self):
        try:
            url_match = re.search(
        'at\.ua|usa\.cc|baltazarpresentes\.com\.br|pe\.hu|esy\.es|hol\.es|sweddy\.com|myjino\.ru|96\.lt|ow\.ly', url)
            ip_address = socket.gethostbyname(self.domain)
            ip_match = re.search('146\.112\.61\.108|213\.174\.157\.151|121\.50\.168\.88|192\.185\.217\.116|78\.46\.211\.158|181\.174\.165\.13|46\.242\.145\.103|121\.50\.168\.40|83\.125\.22\.219|46\.242\.145\.98|'
                                '107\.151\.148\.44|107\.151\.148\.107|64\.70\.19\.203|199\.184\.144\.27|107\.151\.148\.108|107\.151\.148\.109|119\.28\.52\.61|54\.83\.43\.69|52\.69\.166\.231|216\.58\.192\.225|'
                                '118\.184\.25\.86|67\.208\.74\.71|23\.253\.126\.58|104\.239\.157\.210|175\.126\.123\.219|141\.8\.224\.221|10\.10\.10\.10|43\.229\.108\.32|103\.232\.215\.140|69\.172\.201\.153|'
                                '216\.218\.185\.162|54\.225\.104\.146|103\.243\.24\.98|199\.59\.243\.120|31\.170\.160\.61|213\.19\.128\.77|62\.113\.226\.131|208\.100\.26\.234|195\.16\.127\.102|195\.16\.127\.157|'
                                '34\.196\.13\.28|103\.224\.212\.222|172\.217\.4\.225|54\.72\.9\.51|192\.64\.147\.141|198\.200\.56\.183|23\.253\.164\.103|52\.48\.191\.26|52\.214\.197\.72|87\.98\.255\.18|209\.99\.17\.27|'
                                '216\.38\.62\.18|104\.130\.124\.96|47\.89\.58\.141|78\.46\.211\.158|54\.86\.225\.156|54\.82\.156\.19|37\.157\.192\.102|204\.11\.56\.48|110\.34\.231\.42', ip_address)
            if url_match:
                return -1
            elif ip_match:
                return -1
            return 1
        except:
            return 1
    
    def getFeaturesList(self):
        return self.features
        

In [ ]:
# 1. Importación de Librerías
import csv
import pandas as pd
import numpy as np
import re
import os

# 2. Cargar Archivo de Datos
archivo_datos = "datos_tesis.csv"
if os.path.exists(archivo_datos):
    df = pd.read_csv(archivo_datos)
else:
    print(f"⚠️ Archivo '{archivo_datos}' no encontrado. Se creará uno nuevo.")
    df = pd.DataFrame(columns=["Numero", "Caracteristicas", "Clase", "URL", "Mensaje"])

# 3. Definir Patrón para Validar URLs
pattern = r"^(?:http|ftp)s?://[a-zA-Z0-9-]+(\.[a-zA-Z0-9-]+)+\.bo/?"

# 4. URL a Evaluar
url = "https://snis.minsalud.gob.bo/"

# 5. Validar la URL
if re.match(pattern, url):
    try:
        obj = ExtraccionUrl(url)  # Asegurar que la clase está bien definida
        x = np.array(obj.getFeaturesList()).reshape(1, -1)  # Adaptar tamaño dinámicamente
        print("✅ Características extraídas:", x)

        # 6. Validar que el modelo `gbc` está disponible
        try:
            y_pred = gbc.predict(x)[0]
        except NameError:
            print("⚠️ Error: El modelo `gbc` no está definido.")
            y_pred = -1  # Asignar un valor por defecto

        # 7. Obtener el número de registro
        if 'Numero' in df.columns and not df.empty:
            ultimo_numero = df['Numero'].max() + 1
        else:
            ultimo_numero = 1

        # 8. Clasificación y mensaje de salida
        if y_pred == 1:
            mensaje = "Sitio web seguro"
            clase = -1
        else:
            mensaje = "Sitio web sospechoso detectado"
            clase = 1
        print(f"🔍 Resultado: {mensaje}")

        # 9. Almacenar en CSV
        nueva_fila = [ultimo_numero, list(x.flatten()), clase, url, mensaje]
        df.loc[len(df)] = nueva_fila  # Agregar nueva fila al DataFrame

        df.to_csv(archivo_datos, index=False, sep=";")
        print("✅ Datos guardados correctamente en 'datos_tesis.csv'.")

    except Exception as e:
        print(f"⚠️ Error procesando la URL: {e}")
else:
    print("🚫 La URL no es válida o no pertenece al dominio .bo.")


In [1]:
# 1. Importación de Librerías
import csv
import pandas as pd
import numpy as np
import re
import json

# 2. Cargar URLs desde JSON
with open('url_extractor/urls_bo.json', 'r') as file:
    urls_bo = json.load(file)

# Convertir las URLs en una lista
urls_bo_array = [item['url'] for item in urls_bo]

# 3. Cargar Datos de la Tesis desde CSV
df = pd.read_csv('datos_tesis.csv')

# 4. Inicializar el contador para `Numero`
# Si la columna 'Numero' existe, se obtiene el último valor registrado y se incrementa en 1
if 'Numero' in df.columns:
    contador_numero = df['Numero'].max() + 1
else:
    contador_numero = 1  # Si la columna no existe, se inicia desde 1

# 5. Definir la función para procesar cada URL
def process_url(url, contador_numero):
    """
    Valida la URL, extrae características, predice su seguridad y la almacena en CSV.

    Parámetros:
    - url (str): URL a analizar.
    - contador_numero (int): Contador para asignar un identificador único.

    Retorna:
    - contador_numero actualizado.
    """

    # Validar si la URL tiene un dominio .bo
    pattern = r"^(?:http|ftp)s?://[a-zA-Z0-9-]+(\.[a-zA-Z0-9-]+)+\.bo/?"
    
    if re.match(pattern, url):
        # Crear un objeto de ExtraccionUrl para obtener características de la URL
        obj = ExtraccionUrl(url)
        x = np.array(obj.getFeaturesList()).reshape(1, 30)  # Ajustar formato de los datos
        print(f"🔍 Características de {url}: \n{x}")

        # Realizar la predicción con el modelo `gbc`
        y_pred = gbc.predict(x)[0]

        # Definir mensaje y clasificación según el resultado de la predicción
        if y_pred == 1:
            mensaje = "Sitio web seguro"
            clase = -1
        else:
            mensaje = "Sitio web sospechoso detectado"
            clase = 1
        
        print(mensaje

SyntaxError: incomplete input (2893036410.py, line 58)